In [1]:
import pandas as pd
import numpy as np

def convert_number(x):
    total_stars = 0
    if 'K' in x:
        if len(x) > 1:
            total_stars = float(x.replace('K', '')) * 1000 # convert k to a thousand
    elif 'M' in x:
        if len(x) > 1:
            total_stars = float(x.replace('M', '')) * 1000000 # convert M to a million
    elif 'B' in x:
        total_stars = float(x.replace('B', '')) * 1000000000 # convert B to a Billion
    else:
        total_stars = int(x) # Less than 1000
    
    return int(total_stars)

train = pd.read_csv('credit_card_default_train.csv')
test = pd.read_csv('credit_card_default_test.csv')

fulldataset = [train,test]

for dataset in fulldataset:
    dataset['Gender'] = dataset['Gender'].map({'F':0, 'M':1}).astype(int)
    
for dataset in fulldataset:
    dataset['MARITAL_STATUS'] = dataset['MARITAL_STATUS'].map({'Single':1, 'Married':2, 'Other':3}).astype(int)

for dataset in fulldataset:
    dataset['EDUCATION_STATUS'] = dataset['EDUCATION_STATUS'].map({'High School':1, 'Graduate':2, 'Other':3}).astype(int)
    
for dataset in fulldataset:
    dataset['AGE'] = dataset['AGE'].map({'Less than 30':1, '31-45':2, '46-65':3, 'More than 65':4}).astype(int)
    
train['Balance_Limit_V1'] = train['Balance_Limit_V1'].apply(convert_number)
test['Balance_Limit_V1'] = test['Balance_Limit_V1'].apply(convert_number)

In [2]:
train.head()

,Client_ID,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
0,A2,1000000,1,2,3,2,-1,-1,-1,-1,...,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
1,A3,1000000,0,1,3,1,0,-1,-1,-1,...,43530,80811,124590,151818,46200,43530,80811,942,33666,0
2,A4,100000,0,1,1,2,4,3,2,2,...,0,0,0,0,0,0,0,0,0,1
3,A5,200000,0,2,1,2,2,0,0,0,...,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
4,A6,1000000,0,2,3,2,2,2,0,0,...,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [3]:
from sklearn.svm import SVC

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [4]:
test.loc[(test['PAY_OCT']==test['PAY_NOV']) & (test['PAY_OCT'] == test['PAY_DEC']),'PAY_MEAN'] = test['PAY_OCT']
test.loc[test['PAY_MEAN'].isnull(),'PAY_MEAN'] = round((test['PAY_OCT']+test['PAY_NOV']+test['PAY_DEC'])/3)
test['PAY_MEAN'] =test['PAY_MEAN'].astype(int)

In [5]:
test.head()

,Client_ID,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,PAY_MEAN
0,A20170,500000,1,3,1,2,0,0,0,0,...,311201,351790,352206,13710,12816,195701,53246,17256,12991,0
1,A16887,200000,0,3,1,2,0,0,0,0,...,94181,97221,100275,3756,3842,3890,4620,4620,4620,0
2,A3430,400000,1,3,3,1,-1,-1,2,0,...,11294,9561,993,6930,0,2310,0,2310,40060,0
3,A3696,1500000,1,2,1,2,1,-1,-1,-2,...,0,0,0,5775,5775,0,0,0,0,-2
4,A20474,100000,0,1,3,1,0,0,-2,-2,...,0,0,0,0,0,0,0,0,0,-2


In [6]:
train.loc[(train['PAY_OCT']==train['PAY_NOV']) & (train['PAY_OCT'] == train['PAY_DEC']),'PAY_MEAN'] = train['PAY_OCT']
train.loc[train['PAY_MEAN'].isnull(),'PAY_MEAN'] = round((train['PAY_OCT']+train['PAY_NOV']+train['PAY_DEC'])/3)
train['PAY_MEAN'] =train['PAY_MEAN'].astype(int)

In [7]:
train.head()

,Client_ID,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,...,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT,PAY_MEAN
0,A2,1000000,1,2,3,2,-1,-1,-1,-1,...,162772,-13982,3437,6004,39418,162772,0,538165,0,-1
1,A3,1000000,0,1,3,1,0,-1,-1,-1,...,80811,124590,151818,46200,43530,80811,942,33666,0,-1
2,A4,100000,0,1,1,2,4,3,2,2,...,0,0,0,0,0,0,0,0,1,-1
3,A5,200000,0,2,1,2,2,0,0,0,...,100353,102740,3855,3890,3696,4620,4049,3918,1,0
4,A6,1000000,0,2,3,2,2,2,0,0,...,445271,453899,0,20790,16170,17325,16401,17325,0,0


In [8]:
x_train = train.drop(['Client_ID','NEXT_MONTH_DEFAULT'],axis=1)
y_train = train['NEXT_MONTH_DEFAULT']

In [9]:
x_train.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,PAY_MEAN
0,1000000,1,2,3,2,-1,-1,-1,-1,-1,...,39418,162772,-13982,3437,6004,39418,162772,0,538165,-1
1,1000000,0,1,3,1,0,-1,-1,-1,-1,...,43530,80811,124590,151818,46200,43530,80811,942,33666,-1
2,100000,0,1,1,2,4,3,2,2,-2,...,0,0,0,0,0,0,0,0,0,-1
3,200000,0,2,1,2,2,0,0,0,0,...,97309,100353,102740,3855,3890,3696,4620,4049,3918,0
4,1000000,0,2,3,2,2,2,0,0,0,...,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [10]:
kfold = KFold(n_splits=10,shuffle=True,random_state=0)
clf = SVC()
score = cross_val_score(clf,x_train,y_train,cv=kfold,n_jobs=1,scoring='accuracy')
print(score.mean())

C:\Users\Nimper\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nimper\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nimper\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nimper\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureW

0.7801666666666666


In [11]:
x_train = x_train.drop(['DUE_AMT_JULY','DUE_AMT_AUG','DUE_AMT_SEP','PAID_AMT_JULY','PAID_AMT_AUG','PAID_AMT_SEP','DUE_AMT_OCT','DUE_AMT_NOV','DUE_AMT_DEC','PAID_AMT_OCT','PAID_AMT_NOV','PAID_AMT_DEC'],axis=1)

In [12]:
x_train.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,PAY_MEAN
0,1000000,1,2,3,2,-1,-1,-1,-1,-1,-1,-1
1,1000000,0,1,3,1,0,-1,-1,-1,-1,0,-1
2,100000,0,1,1,2,4,3,2,2,-2,-2,-1
3,200000,0,2,1,2,2,0,0,0,0,0,0
4,1000000,0,2,3,2,2,2,0,0,0,0,0


In [13]:
kfold = KFold(n_splits=10,shuffle=True,random_state=0)
clf = SVC()
score = cross_val_score(clf,x_train,y_train,cv=kfold,n_jobs=1,scoring='accuracy')
print(score.mean())

C:\Users\Nimper\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nimper\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nimper\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nimper\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureW

0.817


In [14]:
clf = SVC()
clf.fit(x_train,y_train)

C:\Users\Nimper\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [16]:
x_test = test.drop(['Client_ID','DUE_AMT_JULY','DUE_AMT_AUG','DUE_AMT_SEP','PAID_AMT_JULY','PAID_AMT_AUG','PAID_AMT_SEP','DUE_AMT_OCT','DUE_AMT_NOV','DUE_AMT_DEC','PAID_AMT_OCT','PAID_AMT_NOV','PAID_AMT_DEC'],axis=1)
prediction = clf.predict(x_test)

In [17]:
submission = pd.DataFrame({
        'Client_ID':test['Client_ID'],
        'NEXT_MONTH_DEFAULT':prediction
    })

submission.to_csv('submission.csv',index=False)

In [18]:
dataset = pd.read_csv('submission.csv')
dataset.head()

,Client_ID,NEXT_MONTH_DEFAULT
0,A20170,0
1,A16887,0
2,A3430,0
3,A3696,0
4,A20474,0
